In [1]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
import numpy as np
from sklearn.metrics import accuracy_score
from nltk.stem.porter import PorterStemmer
import spacy
import os

In [2]:
pd.options.mode.chained_assignment = None
nltk.download('stopwords')

file_prefix = 'bio'
text_field = 'user description'

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/justinpahl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
csvname="filtered_dataset/"+file_prefix+".csv"
data_orig=pd.read_csv(csvname)
data=data_orig.copy()
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
rows=len(data)

In [4]:
def filter_text(in_data):
  data = in_data.copy()
  for i in range (0,rows):
    oldtext=data[text_field][i] 
    newtext=' '.join(re.sub("(@[A-Za-z0-9]+)| (_URL_) | (#[A-Za-z0-9]+)|(\w+:\/\/\S+)"," ",str(oldtext)).split()) # removes hashtags, https links, usernames.
    newtext= re.sub('[^a-zA-Z0-9]'," ",str(newtext)) #removes all other characters other than English Alpha-Numeric Characters
    newtext=newtext.lower()
    newtext=newtext.split()
    data[text_field][i]=newtext
  return data

In [5]:
filtered_data=filter_text(data)
filtered_data

,user description,class
0,"[mother, of, 4, ex, wife, of, 3, battling, hea...",0
1,"[wchs, tv, serves, the, charleston, huntington...",0
2,"[patient, engagement, revenue, cycle, manageme...",0
3,"[home, of, the, minnesota, and, always, live, ...",0
4,"[spectrum, news, 1, offers, the, best, 24, hou...",0
...,...,...
5782,[none],1
5783,[none],1
5784,"[hermit, who, loves, to, read, and, has, 4, do...",1
5785,"[a, well, educated, mind, will, always, have, ...",1


In [6]:
def porterstemming(in_data):
  data = in_data.copy()
  for i in range (0,rows):
    ps=PorterStemmer()
    oldtext=data[text_field][i]
    newtext= [ps.stem(word) for word in oldtext if word not in set(stopwords.words('english'))]
    newtext=' '.join(newtext)
    data[text_field][i]=newtext
  return data

In [7]:
portered_stemmed_data=porterstemming(filtered_data)
portered_stemmed_data

,user description,class
0,mother 4 ex wife 3 battl health inequ misinfor...,0
1,wch tv serv charleston huntington market lates...,0
2,patient engag revenu cycl manag healthcar thou...,0
3,home minnesota alway live free app,0
4,spectrum news 1 offer best 24 hour local news ...,0
...,...,...
5782,none,1
5783,none,1
5784,hermit love read 4 dog like peopl,1
5785,well educ mind alway question answer helen keller,1


In [8]:
def lemmatization(in_data):
  data = in_data.copy()
  spc = spacy.load("en_core_web_sm")
  for i in range (0,rows):
    oldtext = ' '.join(item for item in data[text_field][i])
    doc = spc(oldtext)
    newtext=' '.join([str(token) for token in doc if token not in set(stopwords.words("english"))])
    newtext = re.sub(r'\s+', ' ', newtext)
    data[text_field][i]=newtext
  return data

In [9]:
lemmatization_data=lemmatization(filtered_data)
lemmatization_data

,user description,class
0,mother of 4 ex wife of 3 battling health inequ...,0
1,wchs tv serves the charleston huntington marke...,0
2,patient engagement revenue cycle management he...,0
3,home of the minnesota and always live on the f...,0
4,spectrum news 1 offers the best 24 hour local ...,0
...,...,...
5782,none,1
5783,none,1
5784,hermit who loves to read and has 4 dogs that i...,1
5785,a well educated mind will always have more que...,1


In [10]:
def create_corpus_and_y(data):
  y=np.array(data["class"])
  corpus=[data[text_field][i] for i in range(len(data)) if len(data)!=0]
  return (y,corpus)

In [11]:
y_portered,portered_corpus=create_corpus_and_y(portered_stemmed_data)
y_lemmatization,lemmatization_corpus=create_corpus_and_y(lemmatization_data)

In [12]:
def count_vec(corpus):
  from sklearn.feature_extraction.text import CountVectorizer
  cv = CountVectorizer(max_features = 1500)
  X = cv.fit_transform(corpus).toarray()
  return X

In [13]:
def tf_idf_vec(corpus):
  from sklearn.feature_extraction.text import TfidfVectorizer
  vectorizer = TfidfVectorizer(max_features=1500) # Add min_df, max_df parameters as well
  vectorizer.fit(corpus)
  X = vectorizer.transform(corpus)
  X=X.toarray()
  return X

In [14]:
X_portered=count_vec(portered_corpus)
X_lemmatization=count_vec(lemmatization_corpus)

In [15]:
df_portered = pd.DataFrame(X_portered)
df_portered['class'] = y_portered
df_portered

,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5782,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5783,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5784,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5785,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
df_lemmatization = pd.DataFrame(X_lemmatization)
df_lemmatization['class'] = y_lemmatization
df_lemmatization

,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5782,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5783,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5784,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5785,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
#write processed dataset to .csv
os.makedirs('processed_dataset', exist_ok=True)
df_portered.to_csv('processed_dataset/'+file_prefix+'_portered.csv',index=False) 
df_lemmatization.to_csv('processed_dataset/'+file_prefix+'_lemmatization.csv',index=False) 

In [18]:
#def split_data_train_test_split(X,y):
#  from sklearn.model_selection import train_test_split
#  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
#  return(X_train, X_test,y_train,y_test)

In [19]:
#X_train_cv, X_test_cv,y_train_cv,y_test_cv=split_data_train_test_split(X_cv,y)
#X_train_tf, X_test_tf,y_train_tf,y_test_tf=split_data_train_test_split(X_tf,y)